# House Price Prediction using Linear Regression

`Author:` [Sagar Kanekar](https://github.com/TheShade1551)\
`Date:` 10.December.2024\
`Dataset:`[AMES Housing Dataset](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)


### About Dataset(Meta Data)

#### Context:-
- Following Dataset is a Multivariate Dataset of Collection of Different Houses & their Detailed Descriptions with the Price at Which they were sold.
- This Dataset Contains 79 Attributes (Categorical & Numerical) which provide Diverse Information to Predict the Target Variable i.e. Selling Price of the Respective House.

#### Content:-
- The 79 Attributes in this dataset can be classified as:-
  1. **Zone Attributes**:- Focusing on the Location of Property in context of Surroundings.
  2. **Lot Configurations**:- Regarding Properties of the Lot Specifications.
  3. **House Floors**:- Data about Floorwise Informations(Basement, 1st & 2nd Floor)
  4. **Amenities**:- About Different Aspects of House other than Living Area viz. Fireplaces, Garage, Pool, Porches,
  5. **Rooms**:- Information about Living Rooms, Bathrooms & Kitchens within House.
  6. **Selling Data**:- Selling Details, eg. Month, Year, Type of Deal. etc.

### Acknowledgements:-
- The Ames Housing dataset was compiled by Dean De Cock for use in data science education.
- It Provides a Modernized and Expanded version of the often cited Boston Housing dataset.
- The Dataset is Made Available by Kaggle through Open Competition.

### Citation:-
- [Anna Montoya and DataCanary. House Prices - Advanced Regression Techniques.](www.kaggle.com/competitions/house-prices-advanced-regression-techniques/overview/$citation)

---
### Problem Statement:-
- To Implement a Linear Regression Model to Predict the Prices of Houses based on the following:-
  1. Square Footage of House
  2. Number of Bedrooms & Bathrooms

### Procedure:-
- To Get the Designated Features within a Dataframe
- Exploratory Data Analysis
- Handling Missing Values
- Scaling Data(Wherever Necessary)
- Combining Data into 3 Features
  1. Total_Living_Area:- Footage of All Living Area Combined.(All the Floors)
  2. Total_Square_Footage:- Total Area = Living Area + Amenities Area
  3. Rooms:- Total Number of Bedrooms & Bathrooms(considering appropriate weights)
- Multivariate Linear Regression
  1. With Total_Living_Area & Rooms Features
  2. With Total_Square_Footage & Rooms Features
  3. With Plot Area & Rooms Features
- Measuring Accuracy using Evaluation Metrics






# `Import Libraries`

In [ ]:
# Importing Libraries

# Data Manipulation & Analysis.
import pandas as pd
import numpy as np

# Data Visulization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning Models & Utilities
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

# Handling Unnecessary Warnings
import warnings
warnings.filterwarnings('ignore')

#`Load Dataset`

In [ ]:
#Loading Dataset
df = pd.read_csv('AMES Housing Dataset.csv')

In [ ]:
#Understanding Features:- Data types & Non-Null Counts
df.info()

In [ ]:
df.shape

#### Shortlisting Features
As We Undergo the Regression Task, We Can Only Consider the Numerical Features within the Dataset.
- **Living Area Features**:- `TotalBsmtSF`,`1stFlrSF`,`2ndFlrSF`
- (not considering `GrLivArea` as it only provides the living area above ground, which is already mentioned through `1stFlrSF` & `2ndFlrSF`)
- **Amenities**:- `GarageArea`, `WoodDeckSF`, `OpenPorchSF`, `EnclosedPorch`, `3SsnPorch`, `ScreenPorch`, `PoolArea`
- **Rooms**:- `BsmtFullBath`, `BsmtHalfBath`, `FullBath`, `HalfBath`, `BedroomAbvGr`
- **Lot-Area**:- `LotArea` gives the total Lot Area of the House-Plot
- **Selling-Price**:-`SalePrice`, the Target Variable.

Hence, Shortlisting these 17 Features within a new Dataframe.


In [ ]:
# Creating New Dataframe with Selected Features
Data = df[['TotalBsmtSF','1stFlrSF','2ndFlrSF','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','LotArea','SalePrice']]

In [ ]:
Data.sample(10)

#### Concatenating Features
- Concatenating Living Area Features:- `TotalBsmtSF`,`1stFlrSF`,`2ndFlrSF`
- Concatenating All Porch-Area Features into one (as Most Houses have only 1 type of Porch out of given 4)

In [ ]:
#Creating Feature:- LivingArea
Data['LivingArea'] = Data[['TotalBsmtSF','1stFlrSF','2ndFlrSF']].sum(axis=1)
#Creating Feature:- PorchArea
Data['PorchArea'] = Data[['OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch']].sum(axis=1)

#Dropping Already Concatenated Features
Data.drop(['TotalBsmtSF','1stFlrSF','2ndFlrSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch'],axis=1, inplace=True)

In [ ]:
Data.head()

In [ ]:
Data.shape

In [ ]:
#Checking Null Values
[np.count_nonzero(Data[x]) for x in Data.columns]

Analysis from Null Values:- Very Less Non-Zero Cells in Features `PoolArea` & `BsmtHalfBath`.
But We'll Keep them because we are concatenating features yet again.

#### Concatenating Features - Part-2
- Concatenating Rooms Feature:- `BsmtFullBath`,`BsmtHalfBath`,`FullBath`,`HalfBath`,`BedroomAbvGr` with half rooms as half weights.
- Concatenating Amenities Feature:- `GarageArea`,`WoodDeckSF`,`PoolArea`,`PorchArea`


In [ ]:
#Changing Column Datatypes to Float for Weight-wise Room addition
Data = Data.astype({'BsmtFullBath':'float','BsmtHalfBath':'float','FullBath':'float','HalfBath':'float','BedroomAbvGr':'float'})
print(Data.dtypes)

In [ ]:
#Creating Feature:- Rooms
Data['BsmtHalfBath'] = Data['BsmtHalfBath']*0.5
Data['HalfBath'] = Data ['HalfBath']*0.5
Data.head()

In [ ]:
Data['Rooms']= Data[['BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr']].sum(axis=1)
Data.drop(['BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr'],axis=1, inplace=True)
Data.head()

In [ ]:
#Creating Feature:- Amenities
Data['Amenities']= Data[['GarageArea','WoodDeckSF','PoolArea','PorchArea']].sum(axis=1)
Data.drop(['GarageArea','WoodDeckSF','PoolArea','PorchArea'],axis=1, inplace=True)

In [ ]:
Data.head()

In [ ]:
#Reordering the Columns
Data= Data.iloc[:, [0,2,4,3,1]]
Data.head()

In [ ]:
#Checking Null Values Again
[np.count_nonzero(Data[x]) for x in Data.columns]

Analysis of Null Values:- Null Values have been Sufficiently Handled.

# `Exploratory Data Analysis`

In [ ]:
#Analysis of Lot Area
print('Lot-Area Summary Statistics:')
Data['LotArea'].describe()

In [ ]:
Data['LotArea'].min(),Data['LotArea'].max()

In [ ]:
sns.histplot(Data['LotArea'],kde=True, bins=30,color='blue')
plt.title('Distribution of Lot Area')
plt.xlabel('Lot Area')
plt.ylabel('Frequency')
plt.show()

Analysis of Lot-Area:- Highly Left Skewed, Needs Outlier Handling

In [ ]:
#Analysis of Living Area
print('Living Area Summary Statistics:')
Data['LivingArea'].describe()

In [ ]:
sns.histplot(Data['LivingArea'],kde=True,bins=30,color='red')
plt.title('Distribution of Living Area')
plt.xlabel('Living Area')
plt.ylabel('Frequency')
plt.show()

Analysis of Living Area:- Comparitively Normal, Needs a bit of Right Outlier Handling

In [ ]:
#Analysing Amenities
Data['Amenities'].min(),Data['Amenities'].max()

In [ ]:
sns.histplot(Data['Amenities'],kde=True,bins=30,color='green')
plt.title('Distribution of Amenities')
plt.xlabel('Amenities')
plt.ylabel('Frequency')
plt.show()

Analysis of Amenities:- Fairly Normally Distributed.. Needs a bit of Outlier Handling

In [ ]:
#Analysing Rooms:-
sns.histplot(Data['Rooms'],kde=True, bins=30,color='orange')
plt.title('Distribution of Rooms')
plt.xlabel('Rooms')
plt.ylabel('Frequency')
plt.show()

Analysis of Rooms:- Discrete Data, but has central Tendency. No Need of Handling.`

Overall EDA Analysis:-
- Outlier Handling Required in Features `LotArea`,`LivingArea` & `Amenities`

# `Outlier Handling`

In [ ]:
#List of Numerical Columns
num_cols = ['LotArea','LivingArea','Amenities']
for col in num_cols:
  # Calculating Q1, Q3 & IQR
  Q1 = Data[col].quantile(0.25)
  Q3 = Data[col].quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5*IQR
  upper_bound = Q3 + 1.5*IQR

  # identify Outliers
  Outliers = Data[(Data[col]< lower_bound) | (Data[col]> upper_bound)]
  print(f"Column: {col}")
  print(f"Number of Outliers: {Outliers.shape[0]}")

  # Visulaisation
  sns.boxplot(x=Data[col],color='orange')
  plt.title(f'Boxplot of {col}')
  plt.xlabel(col)
  plt.show()

  # Handle Outliers: Remove Rows with Outliers
  Data = Data[(Data[col] >= lower_bound) & (Data[col] <= upper_bound)]
  print(f"After Handling Outliers, Shape of Data: {Data.shape}")
  print("\n")

In [ ]:
fig,axs= plt.subplots(1,3,figsize=(15,5))

sns.histplot(Data['LotArea'],kde=True, bins=30,color='blue', ax=axs[0])
axs[0].set_title('Distribution of Lot Area')
axs[0].set_xlabel('Lot Area')
axs[0].set_ylabel('Frequency')

sns.histplot(Data['LivingArea'],kde=True,bins=30,color='red', ax=axs[1])
axs[1].set_title('Distribution of Living Area')
axs[1].set_xlabel('Living Area')
axs[1].set_ylabel('Frequency')

sns.histplot(Data['Amenities'],kde=True,bins=30,color='green', ax=axs[2])
axs[2].set_title('Distribution of Amenities')
axs[2].set_xlabel('Amenities')
axs[2].set_ylabel('Frequency')
plt.tight_layout()
plt.show()

Hence All 4 Features have been Normalised.

# `Linear Regression - Model Fitting`

- Train, Test Splits to be done.
- Feature Scaling is Required to have a Equivalent weight of All Features. Hence Using Min-Max Scaling.

In [ ]:
# Splitting Training Data => Train, Test

X = Data.drop('SalePrice', axis=1)
y = Data['SalePrice']

X_train,X_test,y_train,y_test = train_test_split(X ,y,test_size=0.33,random_state=42)

In [ ]:
# Feature Scaling

# Intialise the Scaler
scaler = MinMaxScaler()

# fit & transform the data
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [ ]:
# Fitting Linear Regression Model
model = LinearRegression()
model.fit(X_train_sc, y_train)

# `Evaluation`

In [ ]:
y_pred = model.predict(X_test_sc)

In [ ]:
# Plotting the Best-Fit Line
plt.scatter(y_test,y_pred, alpha=0.5)
plt.xlabel('Actual Prices')
plt.ylabel('Predicted Prices')
plt.title('Actual vs Predicted Prices')

plt.plot(y_test,y_test,color='red',linewidth=2, label='Best-Fit Line')
plt.legend()
plt.show()

In [ ]:
# Evaluation Metrics
rmse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared: {r2}")

# `Conclusion`

### Model Evaluation:-
`RMSE:- 33637.91`
- This Indicates that Data Points are Dispersed Widely Around the Mean Value.

`R-Squared:- 0.74`
- R-squared value above 0.70 or 0.80 is considered good, indicating that a large proportion of the variance in the Feature `SalePrice` is explained by the
Processed Features `LotArea`,`LivingArea`,`Amenities`& `Rooms` in the regression model.

### Conclusion
- Linear Regression Model was Successfully fit on `AMES Housing Dataset` to Predict the `Selling Price` of Houses using `Square Footage` & `Number of Bedrooms & Bathrooms`.